In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,748 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [2,311 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,056 kB]
Get:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal 

In [2]:
# enable files acess in google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://raw.githubusercontent.com/wingylui/housing_price_anaylsis/main/Dataset/perth_housing_price_2021.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("perth_housing_price_2021.csv"), header=True, inferSchema=True, quote="\"", escape="\"")

df.show()

+--------------------+------------+------+--------+---------+------+---------+----------+----------+--------+--------------------+----------------+----------+---------+--------+------------+-----------+--------------------+----------------+----------------+
|             ADDRESS|      SUBURB| PRICE|BEDROOMS|BATHROOMS|GARAGE|LAND_AREA|FLOOR_AREA|BUILD_YEAR|CBD_DIST|         NEAREST_STN|NEAREST_STN_DIST|MONTH_SOLD|YEAR_SOLD|POSTCODE|    LATITUDE|  LONGITUDE|         NEAREST_SCH|NEAREST_SCH_DIST|NEAREST_SCH_RANK|
+--------------------+------------+------+--------+---------+------+---------+----------+----------+--------+--------------------+----------------+----------+---------+--------+------------+-----------+--------------------+----------------+----------------+
|       1 Acorn Place|  South Lake|565000|       4|        2|     2|      600|       160|      2003|   18300|Cockburn Central ...|            1800|         9|     2018|    6164|    -32.1159|  115.84245|Lakeland Senior H...|   

In [5]:
# ensure correct format for each column 
df.printSchema()

root
 |-- ADDRESS: string (nullable = true)
 |-- SUBURB: string (nullable = true)
 |-- PRICE: integer (nullable = true)
 |-- BEDROOMS: integer (nullable = true)
 |-- BATHROOMS: integer (nullable = true)
 |-- GARAGE: integer (nullable = true)
 |-- LAND_AREA: integer (nullable = true)
 |-- FLOOR_AREA: integer (nullable = true)
 |-- BUILD_YEAR: integer (nullable = true)
 |-- CBD_DIST: integer (nullable = true)
 |-- NEAREST_STN: string (nullable = true)
 |-- NEAREST_STN_DIST: integer (nullable = true)
 |-- MONTH_SOLD: integer (nullable = true)
 |-- YEAR_SOLD: integer (nullable = true)
 |-- POSTCODE: integer (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- NEAREST_SCH: string (nullable = true)
 |-- NEAREST_SCH_DIST: double (nullable = true)
 |-- NEAREST_SCH_RANK: integer (nullable = true)



In [6]:
# Create a temporary view of the DataFrame.
df.createOrReplaceTempView("perthHousingPrice")

In [ ]:
# ensure it is the correct df 
spark.sql("SELECT * FROM perthHousingPrice LIMIT 3").show()

+---------------+----------+------+--------+---------+------+---------+----------+----------+--------+--------------------+----------------+----------+---------+--------+------------+-----------+--------------------+----------------+----------------+
|        ADDRESS|    SUBURB| PRICE|BEDROOMS|BATHROOMS|GARAGE|LAND_AREA|FLOOR_AREA|BUILD_YEAR|CBD_DIST|         NEAREST_STN|NEAREST_STN_DIST|MONTH_SOLD|YEAR_SOLD|POSTCODE|    LATITUDE|  LONGITUDE|         NEAREST_SCH|NEAREST_SCH_DIST|NEAREST_SCH_RANK|
+---------------+----------+------+--------+---------+------+---------+----------+----------+--------+--------------------+----------------+----------+---------+--------+------------+-----------+--------------------+----------------+----------------+
|  1 Acorn Place|South Lake|565000|       4|        2|     2|      600|       160|      2003|   18300|Cockburn Central ...|            1800|         9|     2018|    6164|    -32.1159|  115.84245|Lakeland Senior H...|    0.8283385518|            nu

## Collect data for Machine Learning (Kmeans)
---

In [ ]:
# understand the amount of house sold in each year
spark.sql("SELECT YEAR_SOLD, ROUND(COUNT(YEAR_SOLD)/(SELECT COUNT(YEAR_SOLD) FROM perthHousingPrice)*100, 2) AS percentOfHouseSold\
          FROM perthHousingPrice\
          GROUP BY YEAR_SOLD\
          ORDER BY percentOfHouseSold DESC").show()

+---------+------------------+
|YEAR_SOLD|percentOfHouseSold|
+---------+------------------+
|     2018|             16.69|
|     2019|             15.71|
|     2020|             15.63|
|     2017|             14.39|
|     2016|             11.59|
|     2015|              9.06|
|     2014|              5.79|
|     2013|               3.5|
|     2012|              1.83|
|     2011|              1.08|
|     2010|              0.81|
|     2009|              0.72|
|     2007|              0.42|
|     2008|              0.42|
|     2006|               0.4|
|     2005|              0.34|
|     2003|              0.26|
|     2002|              0.25|
|     2004|              0.22|
|     2001|              0.15|
+---------+------------------+
only showing top 20 rows



In [8]:
# query the required columns for machine learning (kmeans)
ML_df = spark.sql("SELECT SUBURB, PRICE, LAND_AREA, FLOOR_AREA, BUILD_YEAR, YEAR_SOLD, CBD_DIST, NEAREST_STN_DIST, NEAREST_SCH_DIST\
                  FROM perthHousingPrice\
                  WHERE YEAR_SOLD >= 2013")
ML_df.show(10)

+------------+------+---------+----------+----------+---------+--------+----------------+----------------+
|      SUBURB| PRICE|LAND_AREA|FLOOR_AREA|BUILD_YEAR|YEAR_SOLD|CBD_DIST|NEAREST_STN_DIST|NEAREST_SCH_DIST|
+------------+------+---------+----------+----------+---------+--------+----------------+----------------+
|  South Lake|565000|      600|       160|      2003|     2018|   18300|            1800|    0.8283385518|
|       Wandi|365000|      351|       139|      2013|     2019|   26900|            4900|     5.524324354|
|     Camillo|287000|      719|        86|      1979|     2015|   22600|            1900|     1.649178183|
|    Bellevue|255000|      651|        59|      1953|     2018|   17900|            3600|     1.571400937|
|   Lockridge|325000|      466|       131|      1998|     2016|   11200|            2000|     1.514921629|
|Mount Richon|409000|      759|       118|      1991|     2013|   27300|            1000|     1.227219194|
|     Hilbert|400000|      386|      

In [9]:
# calculate property age and then drop columns (build year and year sold)
ML_df = ML_df.withColumn("PROPERTY_AGE", ML_df["YEAR_SOLD"] - ML_df["BUILD_YEAR"]).drop("BUILD_YEAR")

ML_df.show(10)

+------------+------+---------+----------+---------+--------+----------------+----------------+------------+
|      SUBURB| PRICE|LAND_AREA|FLOOR_AREA|YEAR_SOLD|CBD_DIST|NEAREST_STN_DIST|NEAREST_SCH_DIST|PROPERTY_AGE|
+------------+------+---------+----------+---------+--------+----------------+----------------+------------+
|  South Lake|565000|      600|       160|     2018|   18300|            1800|    0.8283385518|          15|
|       Wandi|365000|      351|       139|     2019|   26900|            4900|     5.524324354|           6|
|     Camillo|287000|      719|        86|     2015|   22600|            1900|     1.649178183|          36|
|    Bellevue|255000|      651|        59|     2018|   17900|            3600|     1.571400937|          65|
|   Lockridge|325000|      466|       131|     2016|   11200|            2000|     1.514921629|          18|
|Mount Richon|409000|      759|       118|     2013|   27300|            1000|     1.227219194|          22|
|     Hilbert|40000

In [10]:
# save file into google drive
ML_df.write.option("header", True).csv("/content/drive/MyDrive/Colab Notebooks/output_csv/Perth_housing_ML.csv")

## Data for Neural Network

In [7]:
# query the required columns for machine learning (kmeans)
Neural_network_df = spark.sql("SELECT PRICE, SUBURB, LAND_AREA, FLOOR_AREA, BUILD_YEAR, YEAR_SOLD, CBD_DIST, NEAREST_STN_DIST, NEAREST_SCH_DIST\
                  FROM perthHousingPrice\
                  WHERE YEAR_SOLD >= 2013")
Neural_network_df.show(10)

+------+------------+---------+----------+----------+---------+--------+----------------+----------------+
| PRICE|      SUBURB|LAND_AREA|FLOOR_AREA|BUILD_YEAR|YEAR_SOLD|CBD_DIST|NEAREST_STN_DIST|NEAREST_SCH_DIST|
+------+------------+---------+----------+----------+---------+--------+----------------+----------------+
|565000|  South Lake|      600|       160|      2003|     2018|   18300|            1800|    0.8283385518|
|365000|       Wandi|      351|       139|      2013|     2019|   26900|            4900|     5.524324354|
|287000|     Camillo|      719|        86|      1979|     2015|   22600|            1900|     1.649178183|
|255000|    Bellevue|      651|        59|      1953|     2018|   17900|            3600|     1.571400937|
|325000|   Lockridge|      466|       131|      1998|     2016|   11200|            2000|     1.514921629|
|409000|Mount Richon|      759|       118|      1991|     2013|   27300|            1000|     1.227219194|
|400000|     Hilbert|      386|      

In [ ]:
# calculate property age and then drop columns (build year and year sold)
Neural_network_df = Neural_network_df.withColumn("PROPERTY_AGE", Neural_network_df["YEAR_SOLD"] - Neural_network_df["BUILD_YEAR"]).drop("BUILD_YEAR")

Neural_network_df.show(10)

+------+---------+----------+---------+--------+----------------+----------------+------------+-----------+------------+
| PRICE|LAND_AREA|FLOOR_AREA|YEAR_SOLD|CBD_DIST|NEAREST_STN_DIST|NEAREST_SCH_DIST|    LATITUDE|  LONGITUDE|PROPERTY_AGE|
+------+---------+----------+---------+--------+----------------+----------------+------------+-----------+------------+
|565000|      600|       160|     2018|   18300|            1800|    0.8283385518|    -32.1159|  115.84245|          15|
|365000|      351|       139|     2019|   26900|            4900|     5.524324354|-32.19347028|115.8595535|           6|
|287000|      719|        86|     2015|   22600|            1900|     1.649178183|-32.12057813| 115.993579|          36|
|255000|      651|        59|     2018|   17900|            3600|     1.571400937|-31.90054675|116.0380088|          65|
|325000|      466|       131|     2016|   11200|            2000|     1.514921629|   -31.88579|  115.94778|          18|
|409000|      759|       118|   

In [ ]:
# save groupby df into google drive
Neural_network_df.write.option("header", True).csv("/content/drive/MyDrive/Colab Notebooks/output_csv/Perth_housing_Neural.csv")

## Collect data for Tableau Data Visualisation
---

In [ ]:
# query the required columns for Data Visualisation
visual_df = spark.sql("SELECT SUBURB, PRICE, LAND_AREA, FLOOR_AREA, BUILD_YEAR, YEAR_SOLD\
                      FROM perthHousingPrice\
                      WHERE YEAR_SOLD >= 2013")
visual_df.show(10)

+------------+------+---------+----------+----------+---------+
|      SUBURB| PRICE|LAND_AREA|FLOOR_AREA|BUILD_YEAR|YEAR_SOLD|
+------------+------+---------+----------+----------+---------+
|  South Lake|565000|      600|       160|      2003|     2018|
|       Wandi|365000|      351|       139|      2013|     2019|
|     Camillo|287000|      719|        86|      1979|     2015|
|    Bellevue|255000|      651|        59|      1953|     2018|
|   Lockridge|325000|      466|       131|      1998|     2016|
|Mount Richon|409000|      759|       118|      1991|     2013|
|     Hilbert|400000|      386|       132|      2014|     2016|
|     Waikiki|370000|      468|       158|      2013|     2019|
|   Hamersley|565000|      875|       168|      1983|     2020|
|   Bayswater|685000|      552|       126|      1999|     2019|
+------------+------+---------+----------+----------+---------+
only showing top 10 rows



In [ ]:
# calculate property age and then drop columns (build year and year sold)
visual_df = visual_df.withColumn("PROPERTY_AGE", visual_df["YEAR_SOLD"] - visual_df["BUILD_YEAR"]).drop("BUILD_YEAR", "YEAR_SOLD")

visual_df.show(10)

+------------+------+---------+----------+------------+
|      SUBURB| PRICE|LAND_AREA|FLOOR_AREA|PROPERTY_AGE|
+------------+------+---------+----------+------------+
|  South Lake|565000|      600|       160|          15|
|       Wandi|365000|      351|       139|           6|
|     Camillo|287000|      719|        86|          36|
|    Bellevue|255000|      651|        59|          65|
|   Lockridge|325000|      466|       131|          18|
|Mount Richon|409000|      759|       118|          22|
|     Hilbert|400000|      386|       132|           2|
|     Waikiki|370000|      468|       158|           6|
|   Hamersley|565000|      875|       168|          37|
|   Bayswater|685000|      552|       126|          20|
+------------+------+---------+----------+------------+
only showing top 10 rows



In [ ]:
# calculate price/floor (price per square meter)
visual_df = visual_df.withColumn("PRICE/FLOOR($/m2)", visual_df["PRICE"]/visual_df["FLOOR_AREA"])

# calculate price/floor (price per square meter)
visual_df = visual_df.withColumn("PRICE/LAND($/m2)", visual_df["PRICE"]/visual_df["LAND_AREA"])

# calculate floor/land
visual_df = visual_df.withColumn("Land/Floor", visual_df["LAND_AREA"]/visual_df["FLOOR_AREA"])

visual_df.show(10)

+------------+------+---------+----------+------------+------------------+------------------+------------------+
|      SUBURB| PRICE|LAND_AREA|FLOOR_AREA|PROPERTY_AGE| PRICE/FLOOR($/m2)|  PRICE/LAND($/m2)|        Land/Floor|
+------------+------+---------+----------+------------+------------------+------------------+------------------+
|  South Lake|565000|      600|       160|          15|           3531.25| 941.6666666666666|              3.75|
|       Wandi|365000|      351|       139|           6|2625.8992805755397|  1039.88603988604|2.5251798561151078|
|     Camillo|287000|      719|        86|          36|3337.2093023255816| 399.1655076495132|  8.36046511627907|
|    Bellevue|255000|      651|        59|          65| 4322.033898305085|391.70506912442397|11.033898305084746|
|   Lockridge|325000|      466|       131|          18| 2480.916030534351| 697.4248927038626|3.5572519083969465|
|Mount Richon|409000|      759|       118|          22| 3466.101694915254|  538.866930171278| 6.

In [ ]:
visual_grp_df = visual_df.groupby("SUBURB").avg("PRICE", "LAND_AREA", "FLOOR_AREA", "PROPERTY_AGE", "PRICE/FLOOR($/m2)", 
                                                "PRICE/LAND($/m2)", "Land/Floor")
visual_grp_df.show()

+----------------+------------------+------------------+------------------+------------------+----------------------+---------------------+------------------+
|          SUBURB|        avg(PRICE)|    avg(LAND_AREA)|   avg(FLOOR_AREA)| avg(PROPERTY_AGE)|avg(PRICE/FLOOR($/m2))|avg(PRICE/LAND($/m2))|   avg(Land/Floor)|
+----------------+------------------+------------------+------------------+------------------+----------------------+---------------------+------------------+
|   Hamilton Hill| 500102.5641025641|  671.025641025641|136.98717948717947| 41.51282051282051|    3854.6552146180593|     899.351120703826| 5.382238510236602|
|   Watermans Bay|        1196421.25|            647.35|          209.0125|26.592105263157894|     6203.204544990847|   2127.9431746909568|3.5112214447619374|
|       Hazelmere| 627307.6923076923|4093.5384615384614|160.65384615384616| 37.69230769230769|     4309.490785511784|   419.78920035627203|27.447816021753326|
|    Piara Waters| 518168.2242990654|488.33644

In [ ]:
# save file into google drive
visual_df.write.option("header", True).csv("/content/drive/MyDrive/Colab Notebooks/output_csv/Perth_housing_visual.csv")

In [ ]:
# save groupby df into google drive
visual_grp_df.write.option("header", True).csv("/content/drive/MyDrive/Colab Notebooks/output_csv/Perth_housing_visual_grp.csv")